<H1>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;DUYGU ANALİZİ PROJESİ</H1>

#### Bu projede; Kaggle platformundan aldığım, Hepsiburada, Trendyol ve N11 gibi  e-ticaret sitelerinden çekilen Türkçe ürün yorumları  kullanılarak bir duygu analizi çalışması yapılmıştır.

In [1]:
import numpy as np
import pandas as pd
import string
import re
import nltk
import snowballstemmer
# nltk.download('stopwords')
from nltk.corpus import stopwords
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv("e-ticaret_urun_yorumlari.csv",delimiter=';')
df.head(20)
# 0: olumsuz
# 1: olumlu
# 2: nötr

,Metin,Durum
0,evet anlatıldığı gibi,1
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1
3,Daha öncede aynısını almıştım çok güzel ve kal...,1
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1
5,ürün gerçekten çok güzel,1
6,Ürün beklediğimden güzel çıktı gayet kullanışl...,1
7,güzel makina tavsiye ederim,1
8,tavsiye edebileceğim çok güzel bir makina,1
9,ürün geldiğinde şarjı vardı. ilk lullanım önce...,1


In [3]:
df['Durum'].value_counts()

0    6978
1    6799
2    1393
Name: Durum, dtype: int64

In [4]:
len(df)

15170

In [5]:
ornek_cumle= df.Metin[2]
ornek_cumle

'Ürün gayet başarılı sakal kesmede başlık sayısı biraz daha fazla olabilirdi.Hem 0 a yakın aliyor. hem de kirli sakal için 3 numara başlık ideal.'

In [6]:
df['Temiz Metin'] = df['Metin']
#buyuk-kucuk donusumu
df['Temiz Metin'] = df['Temiz Metin'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#noktalama işaretleri
df['Temiz Metin'] = df['Temiz Metin'].str.replace('[^\w\s]','')
#sayılar
df['Temiz Metin'] = df['Temiz Metin'].str.replace('\d','')

sw = stopwords.words('turkish')
df['Temiz Metin'] = df['Temiz Metin'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

In [7]:
df['Temiz Metin'][2]

'ürün gayet başarılı sakal kesmede başlık sayısı biraz fazla olabilirdihem a yakın aliyor kirli sakal numara başlık ideal'

In [8]:
df['Metin'][2]

'Ürün gayet başarılı sakal kesmede başlık sayısı biraz daha fazla olabilirdi.Hem 0 a yakın aliyor. hem de kirli sakal için 3 numara başlık ideal.'

In [9]:
df.head(10)

,Metin,Durum,Temiz Metin
0,evet anlatıldığı gibi,1,evet anlatıldığı
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1,öncede almıştım cihazdan ense sakal tüketmek o...
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1,ürün gayet başarılı sakal kesmede başlık sayıs...
3,Daha öncede aynısını almıştım çok güzel ve kal...,1,öncede aynısını almıştım güzel kaliteli bir ürün
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1,erkek kuaförüyüm ense sıfır sakal traşı uygun ...
5,ürün gerçekten çok güzel,1,ürün gerçekten güzel
6,Ürün beklediğimden güzel çıktı gayet kullanışl...,1,ürün beklediğimden güzel çıktı gayet kullanışl...
7,güzel makina tavsiye ederim,1,güzel makina tavsiye ederim
8,tavsiye edebileceğim çok güzel bir makina,1,tavsiye edebileceğim güzel bir makina
9,ürün geldiğinde şarjı vardı. ilk lullanım önce...,1,ürün geldiğinde şarjı vardı ilk lullanım önces...


In [10]:
stemmer = snowballstemmer.stemmer('turkish')
df["Kök Metin"]=df["Temiz Metin"].apply(lambda x: " ".join([stemmer.stemWord(word) for word in x.split()]))

In [11]:
df.head()

,Metin,Durum,Temiz Metin,Kök Metin
0,evet anlatıldığı gibi,1,evet anlatıldığı,evet anlatıldık
1,Daha öncede almıştım bu cihazdan ense ve sakal...,1,öncede almıştım cihazdan ense sakal tüketmek o...,önce almış cihaz en sakal tüketmek on numar sı...
2,Ürün gayet başarılı sakal kesmede başlık sayıs...,1,ürün gayet başarılı sakal kesmede başlık sayıs...,ür gayet başarıl sakal kesme başlık sayıs bira...
3,Daha öncede aynısını almıştım çok güzel ve kal...,1,öncede aynısını almıştım güzel kaliteli bir ürün,önce aynı almış güzel kalitel bir ür
4,Erkek kuaförüyüm ense ve sıfır sakal traşı içi...,1,erkek kuaförüyüm ense sıfır sakal traşı uygun ...,erkek kuaför en sıfır sakal traş uygu bir ür


In [12]:
df['Metin'][2]

'Ürün gayet başarılı sakal kesmede başlık sayısı biraz daha fazla olabilirdi.Hem 0 a yakın aliyor. hem de kirli sakal için 3 numara başlık ideal.'

In [13]:
df['Temiz Metin'][2]

'ürün gayet başarılı sakal kesmede başlık sayısı biraz fazla olabilirdihem a yakın aliyor kirli sakal numara başlık ideal'

In [14]:
df['Kök Metin'][2]

'ür gayet başarıl sakal kesme başlık sayıs biraz faz olabilirdihe a yak aliyor kirli sakal numar başlık ideal'

In [15]:
df['Kök Metin']

0                                          evet anlatıldık
1        önce almış cihaz en sakal tüketmek on numar sı...
2        ür gayet başarıl sakal kesme başlık sayıs bira...
3                     önce aynı almış güzel kalitel bir ür
4             erkek kuaför en sıfır sakal traş uygu bir ür
                               ...                        
15165                                 ışık seviyes rezalet
15166    hic begenmedi aydinlatma hic yok kapattigi aya...
15167                 gün sonra hoparlör bozul kullanışsız
15168         aşır boğuk bir ses rengi var kuma heme bozul
15169                      kuma kalites verdik para değmez
Name: Kök Metin, Length: 15170, dtype: object

In [16]:
x = df['Kök Metin']
y = df['Durum']
# train test split
train_x, test_x, y_train, y_test = train_test_split(x,y, test_size = 0.1,random_state=42)

In [17]:
print(train_x.shape)

(13653,)


In [18]:
print(test_x.shape)

(1517,)


In [19]:
count_vect = CountVectorizer()
X_train = count_vect.fit_transform(train_x)
X_train

<13653x11449 sparse matrix of type '<class 'numpy.int64'>'
	with 129038 stored elements in Compressed Sparse Row format>

In [20]:
X_train.shape

(13653, 11449)

## Naive Bayes ;

In [21]:
clf = MultinomialNB().fit(X_train,y_train)

In [22]:
X_test = count_vect.transform(test_x)

In [23]:
y_pred = clf.predict(X_test)
for  metin,durum in zip(test_x[:30],y_pred[:]):
    print('%r => %s'% (metin,durum)) 

'mario oynamak alınır' => 1
'piyasa gör uygu fiyatlı güzel bir ür' => 1
'ür ebat ufak olduk ia et oynamak pek keyifli gelmedi' => 0
'harik bir ür' => 1
'ür yanlış gel baya ayakkabıs gel ger gönderecek tam bir saçmalık' => 0
'güzel sakal alıyor kes süper' => 1
'güzel bir ürüncok çabuk gel' => 1
'eş hedi al beğe aynı gittik berber kullanıyomus gönül rahatlık alabilir' => 1
'ür biraz dandik plas olmas köt iş görür' => 0
'ür fiyat gör güzel yurt iç kargo köt adres gelme adres sorunlu diyor korgo gidip al sor ilet elaman ilgis adres sorunlu yazıp geçiyor' => 0
'ür kalites i orjinal paketle güzel satıç ilgilihoş vakit geçirmek ideal' => 1
'ür fiyat gör i süper mario oynamak bil yeter' => 1
'gör kalites' => 0
'fevkalade fevk' => 0
'plastik bir ça kız oynamak istedi felaket kokuyor' => 0
'ür güzel değil el alır almaz ekra çık bozuk ür gönder beklemez sayfa bozuk ür gel demekki oluyor sak almay bnce kullanma ekra ca çık m söylemiş beş gönder bileklik takamadı bil ola para ol' => 0
'ür paketle h

In [24]:
from sklearn.metrics import accuracy_score
accuracy_naive =accuracy_score(y_test,y_pred)
print("Doğruluk oranı : %",accuracy_naive * 100)

Doğruluk oranı : % 87.73895847066578


### Yeni yapılan yorumların duygusunun tahmini;

In [25]:
yeni_yorum = pd.Series("Bedeninizi alabilirsiniz, gerçekten güzel…") #olumlu

yeni_yorum2 = pd.Series("Ben gri aldim kahve rengi yolandi nasil KARISIKLIK olur anlamiyorum biraz daha dikkat ve özen lutfen !!!!") #olumsuz

yeni_yorum3 = pd.Series("biraz büyük ama idare eder") #nötr

In [26]:
yeni_yorum = count_vect.transform(yeni_yorum)
yeni_yorum

<1x11449 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [27]:
clf.predict(yeni_yorum)  

array([1], dtype=int64)

In [28]:
yeni_yorum2 = count_vect.transform(yeni_yorum2)
yeni_yorum2

<1x11449 sparse matrix of type '<class 'numpy.int64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [29]:
clf.predict(yeni_yorum2)

array([0], dtype=int64)

In [30]:
yeni_yorum3 = count_vect.transform(yeni_yorum3)
yeni_yorum3

<1x11449 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [31]:
clf.predict(yeni_yorum3)

array([2], dtype=int64)

## Random Forest ;

In [32]:
model = RandomForestClassifier(n_estimators = 100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [33]:
test_xx = count_vect.transform(test_x)
test_xx

<1517x11449 sparse matrix of type '<class 'numpy.int64'>'
	with 13611 stored elements in Compressed Sparse Row format>

In [34]:
test_xx = test_xx.toarray()
test_xx.shape

(1517, 11449)

In [35]:
test_predict = model.predict(test_xx)
accuracy_forest = accuracy_score(y_test, test_predict)
print("Doğruluk oranı : %",accuracy_forest* 100)

Doğruluk oranı : % 88.00263678312459


In [36]:
model.predict(yeni_yorum)

array([1], dtype=int64)

In [37]:
model.predict(yeni_yorum2)

array([0], dtype=int64)

In [38]:
model.predict(yeni_yorum3)

array([0], dtype=int64)